# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine

import sglang as sgl
import asyncio

llm = sgl.Engine(model_path="meta-llama/Meta-Llama-3.1-8B-Instruct")

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:00,  4.77it/s]


Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.78it/s]


Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:01<00:00,  1.35it/s]


Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.21it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.36it/s]



...

 74%|███████▍  | 17/23 [00:04<00:01,  5.22it/s]

 78%|███████▊  | 18/23 [00:04<00:00,  5.20it/s]

 87%|████████▋ | 20/23 [00:04<00:00,  5.16it/s]

 96%|█████████▌| 22/23 [00:05<00:00,  5.17it/s]

100%|██████████| 23/23 [00:05<00:00,  4.35it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Emma and I am a teacher. I'm here today to talk about the importance of promoting literacy in our schools and communities.
Reading is a fundamental skill that is essential for academic success and personal growth. It's not just about reading books, but also about developing critical thinking, problem-solving, and communication skills.
As a teacher, I've seen firsthand the impact that literacy can have on a child's life. A child who struggles with reading may feel frustrated and disengaged, while a child who is a confident reader may feel empowered and motivated.
That's why it's so important for us to promote literacy in our schools and communities.
Prompt: The president of the United States is
Generated text:  a key figure in the country's political landscape, with both significant powers and great responsibility. The president serves as the head of state and the head of government, and is both the commander-in-chief of the military and the ch

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing synchronous streaming generation ===")

for prompt in prompts:
    print(f"\nPrompt: {prompt}")
    print("Generated text: ", end="", flush=True)

    for chunk in llm.generate(prompt, sampling_params, stream=True):
        print(chunk["text"], end="", flush=True)
    print()


=== Testing synchronous streaming generation ===

Prompt: Hello, my name is
Generated text: 

 Rach

ael

 and

 I

...

 and

 organizations

 will

 need

 to

### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Hello, my name is
Generated text:  Laura and I am a 25-year-old graduate student in pursuit of a master’s degree in marine science. I am originally from Florida, where I grew up surrounded by the ocean and developed a deep passion for the marine ecosystem. During my undergraduate studies, I became fascinated with the study of marine ecology, particularly in the realm of coral reefs and their associated biodiversity.
My current research focuses on the effects of climate change on coral reefs, specifically in the Caribbean. I am investigating how increased sea surface temperature and ocean acidification impact coral growth and recruitment. My ultimate goal is to contribute to the development of effective conservation strategies for coral reefs, which

Prompt: The capital of France is
Generated text:  one of the most popular tourist destinations in the world, and for good reason. The City of Light, as it is known, is famous for its stunning architecture, rich history, and vibrant

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        generator = await llm.async_generate(prompt, sampling_params, stream=True)
        async for chunk in generator:
            print(chunk["text"], end="", flush=True)
        print()


asyncio.run(main())


=== Testing asynchronous streaming generation ===

Prompt: Hello, my name is
Generated text: 

 Sean

 (

aka

 Lux

...

 bandwidth

 requirements

 associated

 with

 sending

In [6]:
llm.shutdown()